In [1]:
import sys
pl_path = "/mnt/c/Users/mm851/PycharmProjects/ProteinLearning"
if pl_path not in sys.path:
    sys.path.append(pl_path)

In [2]:
from pyrosetta import *
import numpy as np
from einops import repeat
import torch
init()
atoms = "N CA C".split(" ")
seq = "ARNDCQEGHILKMFPSTWYV"
pose = pose_from_sequence(seq)
b,n,a = 2,len(seq),len(atoms)
coords = torch.zeros(n,a,3)
for i in range(len(seq)):
    for a_idx,a in enumerate(atoms):
        crds = np.array(list(pose.residue(i+1).atom(a).xyz()))
        coords[i,a_idx] = torch.tensor(crds)
b,n,a = 1,len(seq),len(atoms)
coords = repeat(coords, "n a c -> b n a c",b=b)                            

PyRosetta-4 2020 [Rosetta PyRosetta4.Release.python38.linux 2020.28+release.8ecab77aa50ac1301efe53641e07e09ac91fee3b 2020-07-07T16:41:06] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python38.linux r260 2020.28+release.8ecab77 8ecab77aa50ac1301efe53641e07e09ac91fee3b http://www.pyrosetta.org 2020-07-07T16:41:06
core.init: command: PyRosetta -ex1 -ex2aro -database /usr/local/lib/python3.8/dist-packages/pyrosetta-2020.28+release.8ecab77-py3.8-linux-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=1445208817 seed_offset=0 real_seed=1445208817
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=1445208817 RG_type=mt19937
core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard re

In [3]:
from protein_learning.networks.loss.coord_loss import TMLoss
from einops import repeat, rearrange
from pytorch3d.transforms import random_rotation
apply_rot = lambda rot, x : torch.einsum("r c, b n c-> b n r", rot, x)
tm_loss = TMLoss()
real_coords = repeat(coords[:1,:,1,],"b n c -> b (n m) c",m=5)
for sigma in torch.linspace(0,1,5):
    noisy_coords = real_coords + torch.randn_like(real_coords)*sigma
    noisy_coords = apply_rot( random_rotation(), noisy_coords)
    l1 = -tm_loss(real_coords,noisy_coords, align=True)
    l2 = -tm_loss(real_coords,noisy_coords, align=False)
    print("sigma",sigma,"aligned loss :",l1,"unaligned loss",l2)

sigma tensor(0.) aligned loss : tensor(1.) unaligned loss tensor(0.1775)
sigma tensor(0.2500) aligned loss : tensor(0.9853) unaligned loss tensor(0.0568)
sigma tensor(0.5000) aligned loss : tensor(0.9493) unaligned loss tensor(0.0989)
sigma tensor(0.7500) aligned loss : tensor(0.9021) unaligned loss tensor(0.1451)
sigma tensor(1.) aligned loss : tensor(0.8444) unaligned loss tensor(0.0573)


/home/mmcpartlon/.local/lib/python3.8/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [13]:
from protein_learning.networks.loss.coord_loss import FAPELoss
apply_rot = lambda rot, x : torch.einsum("r c, b n a c-> b n a r", rot, x)
fape = FAPELoss()
real_coords = coords
for sigma in torch.linspace(0,0.3,5):
    noisy_coords = real_coords + torch.randn_like(real_coords)*sigma
    noisy_coords = apply_rot(random_rotation(), noisy_coords)
    l1 = fape(real_coords,noisy_coords)
    print("sigma",sigma,"fape:",l1)

sigma tensor(0.) fape: tensor(1.2406e-05)
sigma tensor(0.0750) fape: tensor(0.2108)
sigma tensor(0.1500) fape: tensor(0.2571)
sigma tensor(0.2250) fape: tensor(0.5007)
sigma tensor(0.3000) fape: tensor(0.5795)


In [14]:
from protein_learning.networks.common.rigid import Rigids

In [17]:
bb_rigid = Rigids.RigidFromBackbone(coords)


In [24]:
from einops import rearrange, repeat
rep_coords = repeat(coords[:,:,1], "b n c -> b () n c")
print(rep_coords.shape)
tmp = bb_rigid.apply_inverse(rep_coords)


torch.Size([1, 1, 20, 3])


In [25]:
torch.norm(tmp,dim=-1)


tensor([[[ 0.0000,  3.8021,  7.2856, 10.9825, 14.5712, 18.2465, 21.8568,
          25.5229, 29.1424, 32.8033, 36.4281, 40.0857, 43.7137, 47.3690,
          50.9993, 54.6529, 58.2849, 61.9373, 65.5705, 69.2219],
         [ 3.8021,  0.0000,  3.8021,  7.2856, 10.9825, 14.5712, 18.2465,
          21.8568, 25.5229, 29.1424, 32.8033, 36.4280, 40.0857, 43.7137,
          47.3690, 50.9993, 54.6529, 58.2849, 61.9373, 65.5705],
         [ 7.2856,  3.8021,  0.0000,  3.8021,  7.2856, 10.9825, 14.5712,
          18.2466, 21.8568, 25.5229, 29.1424, 32.8033, 36.4281, 40.0857,
          43.7137, 47.3690, 50.9993, 54.6529, 58.2849, 61.9373],
         [10.9825,  7.2856,  3.8021,  0.0000,  3.8021,  7.2856, 10.9825,
          14.5712, 18.2466, 21.8568, 25.5229, 29.1424, 32.8033, 36.4281,
          40.0857, 43.7137, 47.3690, 50.9993, 54.6529, 58.2849],
         [14.5712, 10.9825,  7.2856,  3.8021,  0.0000,  3.8021,  7.2856,
          10.9825, 14.5712, 18.2466, 21.8568, 25.5229, 29.1424, 32.8033,
          